In [63]:
import tensorflow as tf
import pandas as pd
import numpy as np
import tensorflow.keras.backend as K
from tensorflow.keras.utils import custom_object_scope
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, LSTM, GRU
import yfinance
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

def plot_training_history(history):
    train_loss = history.history['loss']
    test_loss = history.history['val_loss']
    epochs = range(len(train_loss))

    # plot loss
    plt.clf()
    fig = plt.figure()
    plt.plot(train_loss, label='train_loss')
    plt.plot(test_loss, label='test_loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend(loc='best')
    plt.title('Training and Testing Loss')
    plt.show()


In [64]:
#hyperparamters for the stock to input, and the number of data_samples that are relevant data for each prediction
ticker = 'SBUX'
data_samples = 30

#importing data, and clipping off some of the beginning of the
starbucks = yfinance.Ticker(ticker)
df = starbucks.history(period="240mo")
N = len(df) - len(df) % data_samples
df = df.iloc[-N:]

In [65]:
df

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2004-01-08 00:00:00-05:00,6.495664,6.605793,6.489764,6.564495,13144000,0.0,0.0
2004-01-09 00:00:00-05:00,6.529095,6.605792,6.507463,6.509429,10259600,0.0,0.0
2004-01-12 00:00:00-05:00,6.487799,6.574329,6.479933,6.560563,8904400,0.0,0.0
2004-01-13 00:00:00-05:00,6.552696,6.560562,6.481898,6.511397,9841600,0.0,0.0
2004-01-14 00:00:00-05:00,6.529097,6.656925,6.509430,6.613660,9261600,0.0,0.0
...,...,...,...,...,...,...,...
2023-11-27 00:00:00-05:00,102.290001,103.089996,102.070000,102.360001,7853600,0.0,0.0
2023-11-28 00:00:00-05:00,101.959999,102.050003,100.889999,101.180000,6848100,0.0,0.0
2023-11-29 00:00:00-05:00,101.510002,101.690002,99.529999,99.849998,8428500,0.0,0.0


In [66]:
#trimming off the "Dividends" and "Stock Splits" column
df.reset_index(drop = True, inplace=True)
df = df[["Open","High","Low", "Close", "Volume"]]
df = df.iloc[-len(df):]

In [67]:
df

,Open,High,Low,Close,Volume
0,6.495664,6.605793,6.489764,6.564495,13144000
1,6.529095,6.605792,6.507463,6.509429,10259600
2,6.487799,6.574329,6.479933,6.560563,8904400
3,6.552696,6.560562,6.481898,6.511397,9841600
4,6.529097,6.656925,6.509430,6.613660,9261600
...,...,...,...,...,...
5005,102.290001,103.089996,102.070000,102.360001,7853600
5006,101.959999,102.050003,100.889999,101.180000,6848100
5007,101.510002,101.690002,99.529999,99.849998,8428500
5008,100.059998,100.120003,98.419998,99.300003,11442600


In [68]:
# Initialize the MinMaxScaler
scaler = MinMaxScaler()
df_scaled = scaler.fit_transform(df)

# Convert scaled data back to DataFrame for easy handling
df_scaled = pd.DataFrame(df_scaled, columns=df.columns)

In [69]:
df_scaled

,Open,High,Low,Close,Volume
0,0.030452,0.029964,0.032103,0.032058,0.075228
1,0.030738,0.029964,0.032256,0.031586,0.056019
2,0.030384,0.029694,0.032018,0.032024,0.046994
3,0.030940,0.029576,0.032035,0.031603,0.053235
4,0.030738,0.030402,0.032273,0.032479,0.049373
...,...,...,...,...,...
5005,0.851325,0.856306,0.858506,0.852218,0.039996
5006,0.848497,0.847398,0.848303,0.842115,0.033300
5007,0.844641,0.844315,0.836544,0.830728,0.043825
5008,0.832216,0.830869,0.826947,0.826019,0.063897


In [70]:
#hyperparameters
train_percent = 0.9
features = 5

test_percent = (1-train_percent)


#Splitting the data up into train vs test

#Usually in Time-Series ML models, the most recent data is used as testing data, and less recent data
#is used as the training data
train_N = int((train_percent*len(df_scaled)))
test_N = int((test_percent*len(df_scaled)))

train_data = df_scaled.iloc[0:train_N]
test_data = df_scaled.iloc[-test_N:]

In [71]:
#Splitting the train and test dataframes, into data and labels

x_train=[]
y_train=[]

x_test = []
y_test = []


#sliding glass window, we will have `data_samples - len(train_data)` datapoints, meaning we will get much more out of the data
for i in range(data_samples, len(train_data)):
    #[i - data_samples:i+1] represents the current window

    #the first [i - data_samples:i] points are added to the train data (i is not inclusive)
    x_train.append(train_data.iloc[i - data_samples:i].values)
    #add the last value, [i], to the test
    y_train.append(train_data.iloc[i])
    #now we have added a sequence of data to x_train and the label to y_train which is the next datapoint coming after the last datapoint from x_train


#same idea as above
for i in range(data_samples, len(test_data)):
    x_test.append(test_data.iloc[i - data_samples:i].values)
    y_test.append(test_data.iloc[i])

#using np.arrays for tensorflow
x_train = np.array(x_train)
y_train = np.array(y_train)
x_test = np.array(x_test)
y_test = np.array(y_test)

In [72]:
print(x_test.shape)
print(y_test.shape)

(470, 30, 5)
(470, 5)


In [ ]:
#hyperparameters
learning_rate = 0.001
momentum = 0.1
epochs = 200


batch_size = data_samples

model = tf.keras.Sequential([
    GRU(50, activation='tanh', input_shape=(data_samples,features), return_sequences=False),
    Dense(features)
])

#optimizer1 = tf.keras.optimizers.Adam(learning_rate=learning_rate) #alternative optimizer
optimizer2 = tf.keras.optimizers.SGD(learning_rate = learning_rate, momentum=momentum)

#loss function so that the SGD will optimize to get the best possible prediction for "Close"
def custom_close_mae(y_true, y_pred):
    close_true = y_true[3]
    close_pred = y_pred[3]
    return K.abs(close_true - close_pred)

model.summary()


with custom_object_scope({'custom_close_mae': custom_close_mae}):
    model.compile(optimizer=optimizer2, loss='custom_close_mae')
    train_history = model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(x_test, y_test))

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru_2 (GRU)                 (None, 50)                8550      
                                                                 
 dense_2 (Dense)             (None, 5)                 255       
                                                                 
Total params: 8805 (34.39 KB)
Trainable params: 8805 (34.39 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/200
150/150 [==============================] - 3s 12ms/step - loss: 0.1653 - val_loss: 0.3765
Epoch 2/200
150/150 [==============================] - 1s 10ms/step - loss: 0.1031 - val_loss: 0.3372
Epoch 3/200
150/150 [==============================] - 1s 10ms/step - loss: 0.1097 - val_loss: 0.2711
Epoch 4/200
150/150 [==============================] - 2s 10ms/step - loss: 0.0934 - val_loss: 0.2211
Epoch 5/

In [ ]:
plot_training_history(train_history)

In [ ]:
predicted_test = model.predict(x_test)
predicted_train = model.predict(x_train)

In [ ]:
# Create a temporary array with the same shape as your original scaled data
temp_predicted = np.zeros((predicted_test.shape[0], 5))  # 5 features
temp_actual = np.zeros((y_test.shape[0], 5))        # Adjust the test_y shape accordingly

# Copy the 'Close' price predictions and actuals into the temporary arrays
temp_predicted[:, 3] = predicted_test[:, 3]
temp_actual[:, 3] = y_test[:, 3]

# Now, apply inverse_transform on these temporary arrays
predicted_close_prices = scaler.inverse_transform(temp_predicted)[3]
actual_close_prices = scaler.inverse_transform(temp_actual)[3]

# Calculate MAE in dollar terms
mae_in_dollars = np.mean(np.abs(actual_close_prices - predicted_close_prices))

print(f"Model Mean Absolute Error in Dollar Terms: {mae_in_dollars}")

In [ ]:
def plot_prediction_history(train, predicted_train, test, predicted_test, start=None, end=None):
    plt.clf()
    fig = plt.figure()

    train = scaler.inverse_transform(train.values)
    test = scaler.inverse_transform(test.values)
    predicted_train = scaler.inverse_transform(predicted_train)
    predicted_test = scaler.inverse_transform(predicted_test)

    num_samples = test.shape[0] - predicted_test.shape[0]
    train_size = train.shape[0]
    features = train.shape[1]

    zero_array_for_train_p = np.full((num_samples, features), np.nan)
    zero_array_for_test = np.full((train_size, features), np.nan)
    zero_array_for_test_p = np.full((train_size + num_samples, features), np.nan)

    predicted_train_graph = np.vstack((zero_array_for_train_p, predicted_train))
    adj_test = np.vstack((zero_array_for_test, test))
    predicted_test_graph = np.vstack((zero_array_for_test_p, predicted_test))

    plt.plot(train[:, 3], label='train_data')
    plt.plot(predicted_train_graph[:, 3], label='train_predicted')

    plt.plot(adj_test[:, 3], label='test_data')
    plt.plot(predicted_test_graph[:, 3], label='test_predicted')

    if start is not None and end is not None:
        plt.xlim(start, end)

    plt.xlabel('Timestep')
    plt.ylabel('Price')
    plt.title('Price vs Prediction')
    plt.legend()
    plt.show()

In [ ]:
plot_prediction_history(train_data, predicted_train, test_data, predicted_test)

In [ ]:
plot_prediction_history(train_data, predicted_train, test_data, predicted_test, 3000, 5000)

In [ ]:
x_test[-1,:,:]

In [ ]:
y_test[-1,:]